In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import itertools as it
import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score

names = ["Tom      ","Jackson","Ishjot ","David  ","Ruvan  ","Austin  ","Theo     ","Josh      ","Vinny*   ","Andrew   ","Oliver   "]

data = np.loadtxt(open("AFAPy.csv"), delimiter=",",skiprows=1)
datat = np.transpose(data)
logit = lm.LogisticRegression(solver="sag",multi_class="multinomial")
linear = lm.LinearRegression()
TTKReg = lm.LinearRegression()
dubs = datat[15]

players = data[0:,1:12]

In [75]:
logit.fit(players, dubs)
print("Multinomial Logistic Regression without Interactions: Predicting Dub Rate by player only")
print("Intercept: \n", logit.intercept_)
print("Coefficients: \n", logit.coef_)
logitInt = logit.intercept_
logitCoefs = logit.coef_
predictions = logit.predict(players)
print("MSE: ", mean_squared_error(dubs, predictions))
print('R^2: %.2f' % r2_score(dubs, predictions))

Multinomial Logistic Regression without Interactions: Predicting Dub Rate by player only
Intercept: 
 [-1.8706942]
Coefficients: 
 [[ 0.63386681  0.03283491  0.58905869  0.03932072  0.29277109  0.23374646
   0.32943745  0.31604072  0.70344651 -0.07534827 -0.1430911 ]]
MSE:  0.190954773869
R^2: -0.24


In [76]:
adjTTK = datat[14]
linear.fit(players, adjTTK)
print("General Linear Regression without Interactions: Predicting AdjTTK by players only")
print("Intercept: \n", linear.intercept_)
print("Coefficients: \n", linear.coef_)
TTKInt = linear.intercept_
TTKCoefs = linear.coef_
predictions = linear.predict(players)
print("MSE: ", mean_squared_error(adjTTK, predictions))
print('R^2: %.2f' % linear.score(players, adjTTK))

General Linear Regression without Interactions: Predicting AdjTTK by players only
Intercept: 
 6.69628915589
Coefficients: 
 [ 2.43829055  0.46584244  0.67760242 -0.91488196 -1.36686895 -0.77809815
  0.77433593  1.67506098  1.74115932 -1.84500388  1.26048692]
MSE:  37.290163488
R^2: 0.06


In [77]:
AP = datat[12]
TTKReg.fit(players, AP)
print("General Linear Regression without Interactions: Predicting AP by players only")
print("Intercept: \n", TTKReg.intercept_)
print("Coefficients: \n", TTKReg.coef_)
APInt = TTKReg.intercept_
APCoefs = TTKReg.coef_
predictions = TTKReg.predict(players)
print("MSE: ", mean_squared_error(AP, predictions))
print('R^2: %.2f' % TTKReg.score(players, AP))

General Linear Regression without Interactions: Predicting AP by players only
Intercept: 
 17.3212438467
Coefficients: 
 [-4.34443445 -0.97687981 -3.55645728 -0.84715501 -0.53279351 -0.491169
 -2.44858099 -1.96711557 -3.25693142  0.781631   -4.94287027]
MSE:  72.8889953778
R^2: 0.04


In [78]:
print("Name \t\t Multiplicative Effect on Dub Odds Ratio \t Effect on teams AdjTTK \t Effect on AP")
for i in range(0, len(names)):
    print(names[i], "\t", round(np.exp(logitCoefs[0][i]),3),"\t \t \t\t\t\t",round(TTKCoefs[i],3),"\t\t\t\t",round(APCoefs[i],3))
print("Coefficents \t",round(np.exp(logitInt[0]),3),"\t\t\t\t\t\t",round(TTKInt,3),"\t\t\t\t",round(APInt,3))

Name 		 Multiplicative Effect on Dub Odds Ratio 	 Effect on teams AdjTTK 	 Effect on AP
Tom       	 1.885 	 	 				 2.438 				 -4.344
Jackson 	 1.033 	 	 				 0.466 				 -0.977
Ishjot  	 1.802 	 	 				 0.678 				 -3.556
David   	 1.04 	 	 				 -0.915 				 -0.847
Ruvan   	 1.34 	 	 				 -1.367 				 -0.533
Austin   	 1.263 	 	 				 -0.778 				 -0.491
Theo      	 1.39 	 	 				 0.774 				 -2.449
Josh       	 1.372 	 	 				 1.675 				 -1.967
Vinny*    	 2.021 	 	 				 1.741 				 -3.257
Andrew    	 0.927 	 	 				 -1.845 				 0.782
Oliver    	 0.867 	 	 				 1.26 				 -4.943
Coefficents 	 0.154 						 6.696 				 17.321


In [79]:
combos = set(it.combinations(names, 4))
odds = []
probs = []
teamOdds = {}
teamProbs = {}
for team in combos:
    odd = logitInt[0]
    for player in team:
        odd = odd + logitCoefs[0][names.index(player)]
    odds.append(odd)
    probs.append(np.exp(odd)/(1+np.exp(odd)))
    teamOdds[team] = odd
    teamProbs[team] = np.exp(odd)/(1+np.exp(odd))
teamProbs
sorted(teamProbs.items(), key=lambda x: x[1])

[(('Jackson', 'David  ', 'Andrew   ', 'Oliver   '), 0.11743184080617664),
 (('Jackson', 'Austin  ', 'Andrew   ', 'Oliver   '), 0.13912791169502886),
 (('David  ', 'Austin  ', 'Andrew   ', 'Oliver   '), 0.13990654478777204),
 (('Jackson', 'Ruvan  ', 'Andrew   ', 'Oliver   '), 0.14634908791588941),
 (('David  ', 'Ruvan  ', 'Andrew   ', 'Oliver   '), 0.14716122605429066),
 (('Jackson', 'Josh      ', 'Andrew   ', 'Oliver   '), 0.14928017594596482),
 (('David  ', 'Josh      ', 'Andrew   ', 'Oliver   '), 0.15010571950477566),
 (('Jackson', 'Theo     ', 'Andrew   ', 'Oliver   '), 0.15098950680094497),
 (('David  ', 'Theo     ', 'Andrew   ', 'Oliver   '), 0.15182281623867566),
 (('Jackson', 'David  ', 'Austin  ', 'Oliver   '), 0.1534384751790249),
 (('Jackson', 'David  ', 'Ruvan  ', 'Oliver   '), 0.16126327617355285),
 (('Jackson', 'David  ', 'Austin  ', 'Andrew   '), 0.16244596282559404),
 (('Jackson', 'David  ', 'Josh      ', 'Oliver   '), 0.16443552855778243),
 (('Jackson', 'David  ', 'Theo

In [80]:
combos = set(it.combinations(names, 4))
TTKs = []
teamTTKs = {}
for team in combos:
    kills = TTKInt
    for player in team:
        kills = kills + TTKCoefs[names.index(player)]
    TTKs.append(kills)
    teamTTKs[team] = kills
sorted(teamTTKs.items(), key=lambda x: x[1])

[(('David  ', 'Ruvan  ', 'Austin  ', 'Andrew   '), 1.7914362192307633),
 (('Jackson', 'David  ', 'Ruvan  ', 'Andrew   '), 3.0353768118809836),
 (('Jackson', 'Ruvan  ', 'Austin  ', 'Andrew   '), 3.1721606196887504),
 (('Ishjot ', 'David  ', 'Ruvan  ', 'Andrew   '), 3.2471367892896219),
 (('David  ', 'Ruvan  ', 'Theo     ', 'Andrew   '), 3.3438703003054768),
 (('Ishjot ', 'Ruvan  ', 'Austin  ', 'Andrew   '), 3.3839205970973887),
 (('Ruvan  ', 'Austin  ', 'Theo     ', 'Andrew   '), 3.4806541081132436),
 (('Jackson', 'David  ', 'Austin  ', 'Andrew   '), 3.6241476095435003),
 (('David  ', 'Ruvan  ', 'Andrew   ', 'Oliver   '), 3.8300212922930488),
 (('Ishjot ', 'David  ', 'Austin  ', 'Andrew   '), 3.8359075869521377),
 (('David  ', 'Austin  ', 'Theo     ', 'Andrew   '), 3.9326410979679927),
 (('Ruvan  ', 'Austin  ', 'Andrew   ', 'Oliver   '), 3.9668051001008156),
 (('Jackson', 'David  ', 'Ruvan  ', 'Austin  '), 4.1022825414428237),
 (('David  ', 'Ruvan  ', 'Josh      ', 'Andrew   '), 4.24459

In [84]:
combos = set(it.combinations(names, 4))
APs = []
TeamAPs = {}
for team in combos:
    Place = APInt
    for player in team:
        Place = Place + APCoefs[names.index(player)]
    APs.append(Place)
    TeamAPs[team] = Place
sorted(TeamAPs.items(), key=lambda x: x[1])

[(('Tom      ', 'Ishjot ', 'Vinny*   ', 'Oliver   '), 1.2205504291800588),
 (('Tom      ', 'Ishjot ', 'Theo     ', 'Oliver   '), 2.0289008549149958),
 (('Tom      ', 'Theo     ', 'Vinny*   ', 'Oliver   '), 2.3284267144558273),
 (('Tom      ', 'Ishjot ', 'Josh      ', 'Oliver   '), 2.5103662753522631),
 (('Tom      ', 'Josh      ', 'Vinny*   ', 'Oliver   '), 2.8098921348930945),
 (('Ishjot ', 'Theo     ', 'Vinny*   ', 'Oliver   '), 3.1164038908340936),
 (('Tom      ', 'Jackson', 'Ishjot ', 'Oliver   '), 3.5006020423340294),
 (('Ishjot ', 'Josh      ', 'Vinny*   ', 'Oliver   '), 3.5978693112713609),
 (('Tom      ', 'Theo     ', 'Josh      ', 'Oliver   '), 3.6182425606280324),
 (('Tom      ', 'Ishjot ', 'David  ', 'Oliver   '), 3.6303268378541391),
 (('Tom      ', 'Ishjot ', 'Theo     ', 'Vinny*   '), 3.7148397042839583),
 (('Tom      ', 'Jackson', 'Vinny*   ', 'Oliver   '), 3.8001279018748582),
 (('Tom      ', 'David  ', 'Vinny*   ', 'Oliver   '), 3.9298526973949697),
 (('Tom      ', 'Is